# Simplified code for how data is collected through Ping's API. All the code is in the instagram/private_api folder.

This code will not run, but is kept as a reference

In [1]:
from instagram_private_api import Client
from api_requests import (BasicUserInfo, PublicUserInfoRequests, MediaInfoRequests, 
                            InteractWithUsersActions, CollectDataThroughHashtags)

import random
import urllib.parse as parse
import urllib.request as request
import http.cookiejar
from io import BytesIO
import gzip
import json
import pdb

## API Code

In [2]:
def default_headers():
    """
    Creates header information which is passed to url requests in urllib package
    """
    #All of these constants defined in Constants.py
    app_version = '76.0.0.15.395'
    android_version = 24
    android_release = '7.0'
    phone_manufacturer = 'samsung'
    phone_device = 'SM-G930F'
    phone_model = 'herolte'
    phone_dpi = '640dpi'
    phone_resolution = '1440x2560'
    phone_chipset = 'samsungexynos8890'
    version_code = '138226743'
    ig_capabilities = '3brTvw=='
    application_id = '567067343352427'
    FB_HTTP_ENGINE = 'Liger'
    
    USER_AGENT_FORMAT = \
        'Instagram {app_version} Android ({android_version:d}/{android_release}; ' \
        '{dpi}; {resolution}; {brand}; {device}; {model}; {chipset}; en_US; {version_code})'
    user_agent = USER_AGENT_FORMAT.format(**{
            'app_version': app_version,
            'android_version': android_version,
            'android_release': android_release,
            'brand': phone_manufacturer,
            'device': phone_device,
            'model': phone_model,
            'dpi': phone_dpi,
            'resolution': phone_resolution,
            'chipset': phone_chipset,
            'version_code': version_code})
    headers = {
        'User-Agent': user_agent,
        'Connection': 'close',
        'Accept': '*/*',
        'Accept-Language': 'en-US',
        'Accept-Encoding': 'gzip, deflate',
        'X-IG-Capabilities': ig_capabilities,
        'X-IG-Connection-Type': 'WIFI',
        'X-IG-Connection-Speed': '{0:d}kbps'.format(random.randint(1000, 5000)),
        'X-IG-App-ID': application_id,
        'X-IG-Bandwidth-Speed-KBPS': '-1.000',
        'X-IG-Bandwidth-TotalBytes-B': '0',
        'X-IG-Bandwidth-TotalTime-MS': '0',
        'X-FB-HTTP-Engine': FB_HTTP_ENGINE,
    }
    return headers

print('Headers: ')
print(default_headers())

Headers: 
{'User-Agent': 'Instagram 76.0.0.15.395 Android (24/7.0; 640dpi; 1440x2560; samsung; SM-G930F; herolte; samsungexynos8890; en_US; 138226743)', 'Connection': 'close', 'Accept': '*/*', 'Accept-Language': 'en-US', 'Accept-Encoding': 'gzip, deflate', 'X-IG-Capabilities': '3brTvw==', 'X-IG-Connection-Type': 'WIFI', 'X-IG-Connection-Speed': '2417kbps', 'X-IG-App-ID': '567067343352427', 'X-IG-Bandwidth-Speed-KBPS': '-1.000', 'X-IG-Bandwidth-TotalBytes-B': '0', 'X-IG-Bandwidth-TotalTime-MS': '0', 'X-FB-HTTP-Engine': 'Liger'}


In [3]:
def create_opener():
    """
    I think this is the most likely place for the error in my code, I worked backwards to construct opener object
    """
    cookie_jar = http.cookiejar.CookieJar()
    handlers = [
        request.HTTPHandler(),
        request.HTTPSHandler(),
        request.HTTPCookieProcessor(cookie_jar)
    ]

    opener = request.build_opener(*handlers)
    opener.cookie_jar = cookie_jar
    
    return opener

In [4]:
#Directly from Client.py
def _read_response(response):
    """
    Extract the response body from a http response.
    
    :param response:
    :return:
    """
    if response.info().get('Content-Encoding') == 'gzip':
        buf = BytesIO(response.read())
        res = gzip.GzipFile(fileobj=buf).read().decode('utf8')
    else:
        res = response.read().decode('utf8')
    return res

In [5]:
#Inside Client.py:
api_url = 'https://i.instagram.com/api/{version!s}/'

def _call_api(endpoint, query=None, version='v1'):
    """
    I believe every time this method is called in Client.py counts as an API call.
    endpoint param provides url location for API request
    """
    url = '{0}{1}'.format(api_url.format(version=version), endpoint)
    if query:
            url += ('?' if '?' not in endpoint else '&') + parse.urlencode(query)
    print('URL: \n %s'%url)
    headers = default_headers()
    req = request.Request(url, data=None, headers=headers)
    opener = create_opener()
    response = opener.open(req, timeout=15)
    response_content = _read_response(response)
    json_response = json.loads(response_content)
    return json_response

## Testing

In [6]:
"""
Only run once, multiple log ins will cause account ban
TODO: Figure out how to cache auth cookie details. Hints: Can call api.cookie_jar, look into client.py
"""
username = 'cuteanimalzzzz123'
password = 'instabotOP'
api = Client(username, password)

In [7]:
user_id = BasicUserInfo(api, 'ryanlopezzzz').user_id #user_id for ryanlopezzzz

In [8]:
def get_user_info(user_id):
    endpoint = 'users/{user_id!s}/info/'.format(**{'user_id': user_id})
    user_info = _call_api(endpoint)
    return user_info

In [10]:
print(get_user_info(user_id))

URL: 
 https://i.instagram.com/api/v1/users/255272463/info/


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
#Inside endpoints/friendships.py
def user_following(user_id, rank_token, max_id=0):
    #endpoint provides web address for accessing API data
    endpoint = 'friendships/{user_id!s}/following/'.format(**{'user_id': user_id})
    query_params = {'rank_token': rank_token, 'max_id': max_id}
    user_following_info = _call_api(endpoint, query=query_params)
    return user_following_info

In [12]:
rank_token = api.generate_uuid()
print(user_following(user_id, rank_token))

URL: 
 https://i.instagram.com/api/v1/friendships/255272463/following/?rank_token=96e5c8f0-41cc-11ec-9a5d-acde48001122&max_id=0


HTTPError: HTTP Error 403: Forbidden